# Erlang B calculations
## source of truth: http://www.erlang.com/calculator/erlb/


In [182]:
from math import factorial
import pandas as pd
import math

VERBOSE = False

## Return probability of blocking given concurrency and number of lines

In [183]:
########################################################
## Calculate the probability of blocking under Erlang-B model assumptions
########################################################
def ErlangB (concurrency, num_lines):
    InvB = 1.0
    for j in range(1, num_lines+1):
        InvB = 1.0 + InvB * (j/concurrency)
    return (1.0 / InvB)
  

In [184]:
ErlangB(100.0, 101)

0.06972498954186325

## Return Min Number of lines needed to guarantee ErlangB() returns less than pBlock

In [187]:
def numLines (concurrency, pBlock, abs_min_lines = None, incrementer = 1):
   
    concurrency =float (concurrency)
    if abs_min_lines is None:
        abs_min_lines = int (concurrency) + incrementer
    pB_absminlines = ErlangB(concurrency, abs_min_lines)
    pb_minlines =  ErlangB(concurrency, abs_min_lines)
    
    myDF = pd.DataFrame.from_dict({'concur': [concurrency],
                                       'pBlock': [pBlock],
                                       'lines': [abs_min_lines],
                                       'pB': [pb_minlines] }
                                      )
    #print myDF
   
    min_lines = int (abs_min_lines * 1.1)

    ###############################################################
    ##  Loop through the possible min_lines (until calculated pB becomes less then pBlock,
    ##  incrementing them by incrementer
    ###############################################################
    while pb_minlines >= pBlock:
        pb_minlines = ErlangB(concurrency, min_lines)
        mydf = pd.DataFrame.from_dict({'concur': [concurrency],
                                       'pBlock': [pBlock],
                                       'lines': [min_lines],
                                       'pB': [pb_minlines] }
                                      )
        ##print mydf     
        myDF = myDF.append(mydf)
        min_lines = int (min_lines + incrementer)
    
    if VERBOSE:
        return myDF
    else:
        return mydf
        
numLines (100, 0.001)


,concur,lines,pB,pBlock
0,100,128,0.000968,0.001
